In [1]:
from model_UNet import UNet 
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from utils import dice_loss, evaluate_model
from torch.utils.data import DataLoader
from data_into_loaders import get_supervised_data
from data_augmentation import augmentation_all
from ray.tune.search.hyperopt import HyperOptSearch
from ray import tune, air
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
import os

In [ ]:
# Do NOT touch these hyperparameters
img_resize = 64
percentage_validation = 0.1
percentage_test = 0.1

# Change percentage labelled to match the current optimisation parameters.
percentage_labelled = 0.25
max_epochs = 20
#Training params
HYPERPARAMETER_SPACE = {
    'batch_size': tune.choice([16, 32, 64]),
    'gaussian_noise': tune.choice([0.01, 0.1, 1, 10]),
    'lr': tune.choice([1e-2, 1e-3, 1e-4, 1e-5]),
    'lr_decay': tune.choice([0.9, 0.99, 1, None]),
    'depth': tune.choice([3,4,5]),
}

CURRENT_BEST_PARAMS = [{
    'batch_size': 32,
    'gaussian_noise': 1,
    'lr': 1e-3,
    'lr_decay': 0.99,
    'depth': 3
}]

os.system("mkdir Models")
for batch_size in [16, 32, 64]:
    for gaussian_noise in [0.01, 0.1, 1, 10]:
        for lr in [1e-2, 1e-3, 1e-4, 1e-5]:
            for lr_decay in [0.9, 0.99, 1, None]:
                for depth in [3,4,5]:
                    os.system(f"mkdir Models/{batch_size}-{gaussian_noise}-{lr}-{lr_decay}-{depth}")




In [ ]:
train_data, val_data, test_data = get_supervised_data(percentage_labelled, percentage_validation, percentage_test, img_resize=64)

In [14]:
def train(config):
    model = UNet(2, in_channels = 3, depth=config['depth'])
    model = nn.DataParallel(model)
    model.cuda()
    losses, accsTr, IousTr, accsVal, IousVal = [], [], [], [], []
    optimizer = Adam(model.parameters(), lr=config['lr'])
    loss_fn = dice_loss
    s = "/kaggle/working/Models/" + f"{config['batch_size']}-{config['gaussian_noise']}-{config['lr']}-{config['lr_decay']}-{config['depth']}/"
    if config['lr_decay'] is not None:
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['lr_decay'])
    else:
        scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    num_workers = 1

    train_loader = DataLoader(
        train_data,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers = num_workers
    )
    train_loader_eval = DataLoader(
        train_data,
        batch_size=128,
        shuffle=False,
        num_workers = num_workers
    )
    val_loader = DataLoader(
        val_data,
        batch_size=128,
        shuffle=False,
        num_workers = num_workers
    )


    for epoch in range(max_epochs):
        model.train()
        running_loss = 0
        
        for step, data in enumerate(train_loader):
            optimizer.zero_grad()
            imgs, labels = data
            
            imgs_aug, labels_aug = augmentation_all(imgs, labels, config['gaussian_noise'])
            imgs_aug, labels_aug = imgs_aug.cuda(), labels_aug.cuda()
            output = model.forward(imgs_aug)

            loss = loss_fn(output, labels_aug)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        losses.append(running_loss)

        accTr, IouTr = evaluate_model(model, train_loader_eval)  
        accVal, IouVal = evaluate_model(model, val_loader)   

        accsTr.append(accTr)
        IousTr.append(IouTr)
        accsVal.append(accVal)
        IousVal.append(IouVal)
        
        
        
        df = pd.DataFrame({'loss': losses, 'accTr': accsTr, 'IouTr': IousTr, 'accVal': accsVal, 'IouVal': IousVal})
        df.to_csv(s+f"Results.csv")

        session.report({'validation_iou': IouVal, 'running_loss': running_loss, 'train_iou': IouTr, 'validation_accuracy': accVal, 'train_accuracy': accTr})

    
def main(config):
    """
    Main function that defines the hyperparameter search functions, with early-stoppers, HyperOpt
    and resources. Uses HYPERPARAMETER_SPACE as config and a preselected initial starting point
    
    """
    hyperopt_search = HyperOptSearch(
        metric="validation_iou", mode="max"
    )
    hyperopt_search = tune.search.ConcurrencyLimiter(hyperopt_search, max_concurrent=4)
    # Change these to match your GPU setup
    resources_per_trial = {"cpu": .5, "gpu": .5}
    stopper = tune.stopper.TrialPlateauStopper(
        metric = 'validation_iou', 
        std = 0.01, 
        num_results = 3, 
        grace_period = 1,
        mode = 'min',
        metric_threshold=0.7
    )
    tuner = tune.Tuner(
        tune.with_resources(train, resources=resources_per_trial),
        tune_config=tune.TuneConfig(
            search_alg=hyperopt_search,
            num_samples=50
        ),
        param_space=config,
        run_config=air.RunConfig(stop=stopper)
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("validation_iou", "max")
    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation IoU: {}".format(
        best_result.metrics["validation_iou"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["validation_accuracy"]))
    print("Best trial final train IoU: {}".format(
        best_result.metrics["train_iou"]))
    print("Best trial final train accuracy: {}".format(
        best_result.metrics["train_accuracy"]))
    print("Best running loss: {}".format(
        best_result.metrics["running_loss"]))
    
    return best_result


In [ ]:
main(HYPERPARAMETER_SPACE)
